In [1]:
import numpy as np

In [2]:
class gauss_jacobi():
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve( self, x0=None, tolerance=1e-10, max_iter=500):
        """
        Solve Ax = b using the Gauss–Jacobi iterative method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed invertible and preferably diagonally dominant).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        tolerance : float, optional
            Convergence tolerance (default 1e-10).
        max_iter : int, optional
            Maximum number of iterations (default 500).

        Returns
        -------
        x : (n,) ndarray
        Approximate solution.
        num_iter : int
        Number of iterations performed.

        Raises
        ------
        ValueError
        If a zero diagonal entry is found or the method does not converge.
        """

        A = np.array(self.A, dtype=float)#dtype forces elements to be float
        b = np.array(self.b, dtype=float)
        n = len(self.b)

    # Initial guess
        if x0 is None:
            x = np.zeros_like(self.b)#creates a vector of zeros of same length as b
        else:
            x = np.array(x0, dtype=float)

        D = np.diag(self.A) #extracts just the diagonal entries of A as a 1D array
        R = self.A - np.diagflat(D) #creates a 2D diagonal matrix from a 1D array

        for k in range(max_iter):
            x_new = (self.b - np.dot(R, x)) / D #can be seen that this corresponds to the element-wise eqn of G-J
        # Check convergence
            if np.linalg.norm(x_new - x, ord=np.inf) < tolerance:
                return x_new, k 
            x = x_new

        raise ValueError("Jacobi method did not converge within max_iter iterations.")

In [3]:
class GaussSeidel:
    """Iterative Gauss-Seidel solver."""
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve(self, x0=None, tolerance=1e-10, max_iter=500):
        """
        Solve Ax = b using the Gauss–Seidel iterative method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed invertible and preferably diagonally dominant).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        tolerance : float, optional
            Convergence tolerance (default 1e-10).
        max_iter : int, optional
            Maximum number of iterations (default 500).

        Returns
        -------
        x : (n,) ndarray
            Approximate solution.
        num_iter : int
            Number of iterations performed.

        Raises
        ------
        ValueError
            If a zero diagonal entry is found or the method does not converge.
        """

        A = np.array(self.A, dtype=float)  # dtype forces elements to be float
        b = np.array(self.b, dtype=float)
        n = len(self.b)

        # Initial guess
        if x0 is None:
            x = np.zeros_like(self.b)  # creates a vector of zeros of same length as b
        else:
            x = np.array(x0, dtype=float)

        D = np.diag(self.A)  # extracts just the diagonal entries of A as a 1D array

        # Gauss–Seidel uses updated values within the same iteration
        for k in range(max_iter):
            x_old = x.copy()  # keep previous iterate to check convergence
            for i in range(n):
                if D[i] == 0:
                    raise ValueError("Zero diagonal entry encountered; division by zero.")
                # split the row sum into parts before and after i
                sum_before = np.dot(A[i, :i], x[:i])          # uses newly updated components
                sum_after  = np.dot(A[i, i+1:], x_old[i+1:])  # uses previous iteration components
                x[i] = (self.b[i] - sum_before - sum_after) / D[i]

            # Check convergence (∞-norm of difference)
            if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
                return x, k

        raise ValueError("Gauss–Seidel method did not converge within max_iter iterations.")



In [4]:
class ConjugateGradient:
#Conjugate Gradient solver for SPD matrices 
#LSP: can be replaced by any SPD solver
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve(self, x0=None, eps=1e-10, max_iter=None):
        """
        Solve A x = b using the Conjugate Gradient method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed positive definite).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        eps : float, optional
            Convergence tolerance. Iteration stops when ||r_{k+1}|| <= eps * ||r_0||.
        max_iter : int, optional
            Maximum number of iterations (default: size of the system).

        Returns
        -------
        x : (n,) ndarray
            Approximate solution vector.
        num_iter : int
            Number of iterations performed.

        Raises
        ------
        ValueError
            If a breakdown occurs (p^T A p <= 0) or the method does not converge
            within the specified maximum number of iterations.

        Notes
        -----
        This implementation follows the pseudocode:
            1. Initialize x⁰, r⁰ = b - A x⁰, p⁰ = r⁰
            2. For k = 0, 1, 2, ...
                  αₖ = <rᵏ, pᵏ> / <pᵏ, A pᵏ>
                  xᵏ⁺¹ = xᵏ + αₖ pᵏ
                  rᵏ⁺¹ = rᵏ - αₖ A pᵏ
                  If ||rᵏ⁺¹|| ≤ ε ||r⁰||, stop
                  βₖ = - <rᵏ⁺¹, A pᵏ> / <pᵏ, A pᵏ>
                  pᵏ⁺¹ = rᵏ⁺¹ + βₖ pᵏ
        """

        A = np.array(self.A, dtype=float)
        b = np.array(self.b, dtype=float)
        n = b.size

        # Initial guess
        if x0 is None:
            x = np.zeros_like(b)
        else:
            x = np.array(x0, dtype=float)

        # Initial residual and direction
        r = b - np.dot(A, x)
        #Here np.dot dots each row of A with x to get Ax 
        p = r.copy()

        r0_norm = np.linalg.norm(r)
        if r0_norm == 0.0:
            return x, 0

        if max_iter is None:
            max_iter = n

        for k in range(1, max_iter + 1):
            Ap = np.dot(A, p)
            pAp = np.dot(p, Ap)
            if pAp <= 0:
                raise ValueError("Breakdown: p^T A p <= 0 (A may not be PD).")

            # Step size
            alpha = np.dot(r, p) / pAp

            # Update x and r
            x = x + alpha * p
            r_new = r - alpha * Ap

            # Check convergence
            rel_res = np.linalg.norm(r_new) / r0_norm
            if rel_res <= eps:
                return x, k

            # Compute β and update p
            beta = -np.dot(r_new, Ap) / pAp
            p = r_new + beta * p

            # Move to next iteration
            r = r_new

        return x, max_iter